In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import  fisk
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  r2_score
from scipy import integrate

In [3]:
df_population = pd.read_csv('population.csv')
df_population.set_index('year', inplace=True)
df_population = df_population.astype(float)

df_car_stock = pd.read_csv('stock_scooter.csv')
df_car_stock.set_index('year', inplace=True)
df_car_stock['total'] = df_car_stock.sum(axis=1)
df_car_stock = df_car_stock.astype(float)

df_params_gompertz = pd.read_csv('bootstrap_params.csv')
df_params_gompertz.set_index('index', inplace=True)

In [ ]:
list_percentiles = [2.5, 50, 97.5]
pred_stock_total = {}

params_log = {}
for percentile in list_percentiles:
    df_params_log = pd.read_csv('pars_incomeDistribution_' + str(percentile) + '.csv')
    df_params_log.set_index('year', inplace=True)
    params_log[percentile] = {k: v for k, v in zip(df_params_log.index, df_params_log.values)}

def r_gompertz_function(x, a, b, r):
    return r * (np.exp(a * (np.exp(b * x))))


def log_logistic_distribution(x, a, b):
    return fisk.pdf(x, b, 0, a)


def car_stock(x, a, b, alpha, beta, r):
    return log_logistic_distribution(x, a, b) * r_gompertz_function(x, alpha, beta, r)


year_earliest = df_params_log.index[0]
year_start = df_car_stock.index[-1]
year_end = 2050